In [1]:
#%%writefile merge_df.py
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt

plt.style.use('ggplot')


import build_financial_df as fin
import build_results_df as res
merge_df_start = res.results_GE.merge(fin.finance_df, how="inner", left_on="elec-cand", right_on="ELEC-CAND")
merge_df_start.head(5)

#identify Senate (not success)
sen_bool = merge_df_start['office']=="US Senate"
merge_df_start['new_elec_id'] = merge_df_start['elec_id'].str.replace('-00','-SE')

# 8783 races that survived the data cull
num_cand = merge_df_start['new_elec_id'].value_counts()

# 6547 races with 2+ candidates that survived the data cull
num_cand_df = pd.DataFrame()
num_cand_df = num_cand[num_cand>1]
merge_df_cut = merge_df_start[merge_df_start['new_elec_id'].isin(num_cand_df.reset_index()['index'])]

# get top 2 candidates
top2_cand = merge_df_cut.groupby('new_elec_id')['vote%'].nlargest(2)

# make dictionary with values equal to set of names of top 2 candidates 
cand_pair_dct = {}
for pair in top2_cand.index:
    l_name = merge_df_cut.loc[pair[1], 'last_name']
    if pair[0] not in cand_pair_dct:
        cand_pair_dct[pair[0]] = set()
    cand_pair_dct[pair[0]].add(l_name)

    
# list of sets of rematch races
from itertools import chain
rev_dict = {}
for key, value in cand_pair_dct.items():
    rev_dict.setdefault(str(value), set()).add(key)
rematch_list = [values for key, values in rev_dict.items() if len(values) > 1]

# make df of rematches
all_rematch = []
for s in rematch_list:
    for x in s:
        if x not in all_rematch:
            all_rematch.append(x)

index_list = [top2_cand.index[i][1] for i in range(len(top2_cand))]
rematch_df_start = merge_df_start[merge_df_start.index.isin(index_list)]

#write function to get rematch table for each item in rematch_list
def get_rematch_table(idx):
    cand_rematch = merge_df_cut[merge_df_cut.loc[:, 'new_elec_id'].isin(rematch_list[idx])]
    return cand_rematch[['FEC_ID','new_elec_id','year', 'candidate', 'candidatevotes', 'office', 'party', 'state', 'district','INC_STATUS','totalvotes', 'TTL_RECEIPTS', 'TTL_DISB']]

rematch_cut = rematch_df_start[['FEC_ID','new_elec_id','year', 'candidate', 'candidatevotes', 'vote%', 'office', 'party', 'state', 'district','INC_STATUS','totalvotes', 'TTL_RECEIPTS', 'TTL_DISB']]



/Users/Kelly/galvanize/week4/Federal-Election-Funding/build_financial_df.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fin_cut_1['ELEC-CAND'] = fin_cut_1["ELEC_ID"].astype(str) +"-"+ fin_cut_1["LAST_NAME"].astype(str)
/Users/Kelly/galvanize/week4/Federal-Election-Funding/build_results_df.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  results_raw = pd.concat([senate_data_80to18, results_data_80to18], axis=0)
/Users/Kelly/galvanize/week4/Federal-Election-Funding/build_results_df.py:27: SettingWithCopyWarning: 
A value is trying to be set on

In [22]:
#for i in range(len(rematch_list)):
a = rematch_cut[rematch_cut['new_elec_id']==list(rematch_list[10])[0]]
b = rematch_cut[rematch_cut['new_elec_id']==list(rematch_list[10])[1]]
a.set_index('FEC_ID')
b.set_index('FEC_ID')
a['vote%'] - b['vote%']

1842   NaN
1843   NaN
6235   NaN
6237   NaN
Name: vote%, dtype: float64

In [23]:
a

,FEC_ID,new_elec_id,year,candidate,candidatevotes,vote%,office,party,state,district,INC_STATUS,totalvotes,TTL_RECEIPTS,TTL_DISB
6235,H6IN10059,1992-IN-02,1992,Philip R. Sharp,130881,57.079744,US House,democrat,Indiana,2,I,229295,624265.0,623400.0
6237,H6IN10075,1992-IN-02,1992,William G. Frazier,90593,39.509366,US House,republican,Indiana,2,C,229295,176488.0,176034.0


In [19]:
b

,FEC_ID,new_elec_id,year,candidate,candidatevotes,vote%,office,party,state,district,INC_STATUS,totalvotes,TTL_RECEIPTS,TTL_DISB
1842,H6IN10059,1980-IN-10,1980,Philip R. Sharp,103083,53.373823,US House,democrat,Indiana,10,I,193134,141419.0,160827.0
1843,H6IN10075,1980-IN-10,1980,William G. Frazier,90051,46.626177,US House,republican,Indiana,10,NaN,193134,280355.0,279789.0
